# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [1]:
import warnings
warnings.filterwarnings(action='ignore', module='gensim')

import gensim
from gensim.models import Word2Vec, KeyedVectors

In [2]:
# если модель без тэгов
model_path = 'araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'
model = Word2Vec.load(model_path)

In [6]:
# если модель с POS-тэггингом
# model = KeyedVectors.load_word2vec_format(model_path, binary=False)

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [3]:
# "articles_sudbiblioteka"
from tqdm import tqdm
import re

In [4]:
from tqdm import tqdm_notebook as tqdm

In [5]:
def no_spaces(text):
    
    processed_text = text.replace('\n', ' ').replace('\n\n', ' ').replace('ул. ', 'ул.').replace('г. ', 'г.').replace('гор. ', 'гор.').replace('с. ', 'с.')
    return processed_text


# убираем пробел после инициалов перед фамилией
def clear_abbrs(processed_text):
    initials = re.compile(r'[А-Я]{1}\.[А-Я]{1}\. [А-Я][а-яё]+')
    counter = len(initials.findall(processed_text))

    for s in range(counter):
        get_abbrs = initials.search(processed_text)
        i = get_abbrs.span()[0] + 4
        processed_text = processed_text[:i] + processed_text[i+1:]
    return processed_text


# делим текст на предложения при помощи регулярного выражения
def split_text(processed_text):
    
    text_splitted = re.split(r'(\. +[А-Я]{1} *[а-яё]+)', processed_text)
    last_word = re.compile(r'[А-Я]{1} *[а-яё]+')
    normal_sentences = [text_splitted[0] + '.']

    for i in range(1, len(text_splitted), 2):
        if i + 1 <= len(text_splitted)-1:
            beginning = last_word.findall(text_splitted[i])[0]
            normal_sentences.append(beginning + text_splitted[i+1] + '.')
        elif i == len(text_splitted)-1:
            beginning = last_word.findall(text_splitted[i])[0]
            normal_sentences.append(beginning)
    return normal_sentences


def get_sentences(text):
    text = no_spaces(text)
    text = clear_abbrs(text)
    sentences = split_text(text)
    return sentences


# делим текст на куски по n предложений
# (функция принимает на вход список из предложений-строк, полученный на предыдущем шаге)
def split_paragraph(list_of_sentences, n):

    l = len(list_of_sentences)

    n_chunks = []
    chunk = ''

    for i in range(0, l, n):
        for j in range(n):
            if i+j < l:
                chunk += list_of_sentences[i+j] + ' '
            else:
                continue
        n_chunks.append(chunk)
        chunk = ''
    return n_chunks

# main function here
def splitter(text, n):
    """
    :return: split_sentences as a list of strings
    """
    normal_sentences = get_sentences(text)
    split_sentences = split_paragraph(normal_sentences, n)
    return split_sentences

In [6]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [7]:
# this function works with preprocessing func result
def get_w2v_vectors(lemmas_arr, model):
    """Получает вектор документа"""
    if len(lemmas_arr) == 0:
        doc_vec = None
    else:
        vectors = []
        for element in lemmas_arr:
            try:
                vec = model.wv[element]
                # len(vec) this gives us 300
            except KeyError:
                continue
            vectors.append(vec)
        vec_sum = np.zeros(300)
        for v in vectors:
            vec_sum += v
        doc_vec = vec_sum/len(vectors)   # усредненный вектор как опознаватель
    return doc_vec

In [8]:
# test

with open("articles_sudbiblioteka\\article\\3.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    for sentence in splitter(text, 1):
        lemmas_arr = preprocessing(sentence)
        #print(lemmas_arr)
        vec = get_w2v_vectors(lemmas_arr, model)
        print(type(vec))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [9]:
import os
main_dir = 'articles_sudbiblioteka\\article'
file_names = os.listdir(main_dir)

In [10]:
import json
def save_w2v_base(file_names, model):
    """Индексирует всю базу для поиска через word2vec"""
    dc = []
    for name in tqdm(file_names[:1000]):  # testing on a fraction of the data to save time
        with open("articles_sudbiblioteka\\article\\" + name, 'r', encoding='utf-8') as f:
            text = f.read()
            for sentence in splitter(text, 1):
                sentence_vec = get_w2v_vectors(preprocessing(sentence), model)
                dc.append([sentence_vec, sentence, text])
    #with open('saved_w2v_base.json', 'w') as outfile:
    #    json.dump(dc, outfile, ensure_ascii=False)
    return dc

In [11]:
dc = save_w2v_base(file_names, model)

In [13]:
import pandas as pd

In [14]:
pd.DataFrame(dc, columns=['sentence_vec','sentence', 'text']).to_csv("w2v_dc.csv")

In [ ]:
dc = pd.read_csv("w2v_dc.csv")

In [15]:
dc = pd.DataFrame(dc, columns=['sentence_vec','sentence', 'text'])

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [29]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

In [30]:
def train_doc2vec(data):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, min_alpha=0.025, epochs=100, workers=4, dm=1)
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [31]:
file_name = get_tmpfile("d2v_jura")
model_d2v = train_doc2vec(dc['sentence'])
model_d2v.save(file_name)

In [32]:
model_d2v = Doc2Vec.load(file_name)

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [33]:
def get_d2v_vectors(lemmas_arr, model_d2v):
    """Получает вектор документа"""
    # model.infer_vector(["закон", "договор"])
    model_d2v.random.seed(100)  # ensure same results
    if len(lemmas_arr) == None:
        doc_vec = None
    else:
        doc_vec = model_d2v.infer_vector(lemmas_arr)
    return doc_vec

In [22]:
# test

with open("articles_sudbiblioteka\\article\\3.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    for sentence in splitter(text, 1):
        lemmas_arr = preprocessing(sentence)
        #print(lemmas_arr)
        doc_vec = get_d2v_vectors(lemmas_arr, model_d2v)
        #print(doc_vec)
    #print(len(doc_vec))

In [34]:
def save_d2v_base(file_names, model_d2v):
    """Индексирует всю базу для поиска через doc2vec"""
    d = []
    for name in tqdm(file_names[:3000]):
        with open("articles_sudbiblioteka\\article\\" + name, 'r', encoding='utf-8') as f:
            text = f.read()
            for sentence in splitter(text, 1):
                sentence_vec = get_d2v_vectors(preprocessing(sentence, del_stopwords=False), model_d2v)
                d.append([sentence_vec, sentence, text])
    # with open('saved_d2v_base.json', 'w') as outfile:
    #     json.dump(d, outfile, ensure_ascii=False)
    return d

In [35]:
d = save_d2v_base(file_names, model_d2v)

In [36]:
pd.DataFrame(d, columns=['sentence_vec','sentence', 'text']).to_csv("d2v_d.csv")

In [37]:
d = pd.DataFrame(d, columns=['sentence_vec','sentence', 'text'])

In [49]:
d = pd.read_csv("d2v_d.csv")
d['sentence']

0          Судья Высшего Арбитражного Суда Российской Ф...
1        В соответствии с частью 1 статьи 292 Арбитражн...
2        Подающие данное заявление граждане Хорошавин Ю...
3        Оспариваемые судебные акты приняты в отношении...
4        Возвращение к проверке в порядке надзора судеб...
5        Данное заявление не отвечает этим требованиям,...
6        Руководствуясь пунктами 1, 3 части 1 статьи 29...
7                                 Судья Н.А.КСЕНОФОНТОВА. 
8          Судья Высшего Арбитражного Суда Российской Ф...
9        На основании изложенного и руководствуясь стат...
10                                 Судья В.Н.АЛЕКСАНДРОВ. 
11         Высший Арбитражный Суд Российской Федерации ...
12       Заявление принято к производству определением ...
13       Для решения вопроса о наличии оснований для пе...
14       Руководствуясь абзацем 2 части 3 статьи 299 Ар...
15       Председательствующий судья Е.Н.ЗАРУБИНА Судья ...
16         Судья Высшего Арбитражного Суда Российской Ф.

# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

In [ ]:
from collections import defaultdict, Counter
def inverted_index(texts) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    index = defaultdict(list)
    i = 0
    for text in texts:
        for word in text.split(' '):
            if i not in index[word]:
                index[word].append(i)
        i += 1
    return index

In [50]:
from math import log

def score_BM25(qf, dl, avgdl, k1=2.0, b=0.75, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = log((N - n + 0.5)/(n + 0.5)) * (k1 + 1) * qf / (qf + k1 * (1 - b + b * (dl / avgdl)))
    return score

In [ ]:
def compute_sim(query, documents) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
        
    return score


def get_search_result() -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    return

Функция измерения близости между векторами нам пригодится:

In [18]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [43]:
def search_w2v(query, model_vectors, doc_text, model):
    lemmas_query = preprocessing(query, del_stopwords=False)
    query_vector = get_w2v_vectors(lemmas_query, model)
    # print(type(query_vector))
    result = []
    i = 0
    for vec in model_vectors:
        # print(type(vec))
        similar = similarity(query_vector, vec)
        result.append((doc_text[i], similar))
        i += 1
    res = pd.DataFrame(result, columns=['doc_text','similarity'])
    return res.sort_values('similarity', ascending=False)  # sort by similarity

def search_d2v(query, model_vectors, doc_text, model_d2v):
    lemmas_query = preprocessing(query, del_stopwords=False)
    query_vector = get_d2v_vectors(lemmas_query, model_d2v)
    result = []
    i = 0
    for vec in model_vectors:
        similar = similarity(query_vector, vec)
        result.append((doc_text[i], similar))
        i += 1
    res = pd.DataFrame(result, columns=['doc_text','similarity'])
    return res.sort_values('similarity', ascending=False)  # sort by similarity
    

In [44]:
w2v_search = search_w2v('В судебном заседании К. виновным себя не признал.',
                        dc['sentence_vec'], dc['text'], model)
print(w2v_search[:5])

                                               doc_text  similarity
26    ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОНН...    0.845766
5116  ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОНН...    0.821300
7574  ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОНН...    0.809519
9383  ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОНН...    0.804492
5192  ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nОПРЕДЕЛЕНИ...    0.804492


In [45]:
d2v_search = search_d2v('В судебном заседании К. виновным себя не признал.',
                        d['sentence_vec'], d['text'], model_d2v)
print(d2v_search[:5])

                                                doc_text  similarity
26     ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОНН...    1.000000
31680  ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОНН...    0.915588
7574   ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОНН...    0.891737
33123  ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОНН...    0.860533
26385  ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОНН...    0.845447


In [ ]:
def search(query, model_vectors, doc_text, model, search_method):
    if search_method == 'inverted_index':
        search_result = search_inv_index()
    elif search_method == 'word2vec':
        search_result = search_w2v()
    elif search_method == 'doc2vec':
        search_result = search_d2v()
    else:
        raise TypeError('unsupported search method')
    return search_result